In [1]:
import pandas as pd

In [2]:
def get_data(url):
    import requests
    import zipfile
    import io
    response = requests.get(url)
    if response.status_code == 200:
        # Reading zip file from requests response
        zip_file = zipfile.ZipFile(io.BytesIO(response.content))
        # Printing file content in zip
        list_of_files = zip_file.namelist()
        print("ZIP file content:")
        for file_name in list_of_files:
            print(file_name)
        # Extracting file from zip
        zip_file.extractall()
        print("Successfully extracted zip file.")
    else:
        print("Failed to download file. Response:", response.status_code)
    return file_name

In [3]:
# copy url for csv download here
# TODO: the urls need to be updated
urls = [
    "https://www.ebi.ac.uk/chembl/interface_api/delayed_jobs/outputs/DOWNLOAD-MPRDc1E9aDQ-65TODWoM6YAVd4hnMotGX35Ikk5UP4w=/DOWNLOAD-MPRDc1E9aDQ-65TODWoM6YAVd4hnMotGX35Ikk5UP4w=.zip"
]

In [4]:
file_names = []
for url in urls:
    # store file name in file_names
    file_names.append(get_data(url))

ZIP file content:
DOWNLOAD-MPRDc1E9aDQ-65TODWoM6YAVd4hnMotGX35Ikk5UP4w=.csv
Successfully extracted zip file.


In [68]:
# In case you don't want to redownload the csv files online
# run this block to read csv file names available in current directory
# if you ran previous block above, just skip this block
from pathlib import Path
file_names = []
files = Path('./')
for file in files.iterdir():
    if 'DOWNLOAD' in file.name:
        file_names.append(file.name)

In [6]:
def process_data(files):
    import numpy as np
    # initiate data_main, main data frame that will be used to aggregate all csv
    # only use chembl ID, smile, and standard value columns
    data_main = pd.DataFrame({'Molecule ChEMBL ID':[], 'Smiles':[], 'Standard Value':[]})
    # for every csv downloaded, do these following tasks
    for file in files:
        # read data
        data = pd.read_csv(file, sep=';')
        # drop rows if Smiles or Standard Value is null
        removed_null = data.dropna(subset=['Smiles', 'Standard Value'])
        # slice data from initial data frame. only use columns in data_main
        sliced = removed_null[data_main.columns]
        # convert IC50 to pIC50
        # convert IC50 from nM to M
        sliced.loc[:, ['Standard Value']] = sliced[['Standard Value']].apply(lambda x: x*10**-9, axis=1)
        # transform to pIC50
        sliced.loc[:, ['Standard Value']] = -np.log10(sliced[['Standard Value']])
        # join data
        data_main = pd.concat([data_main, sliced])
    # drop duplicates according to Smiles column
    data_main = data_main.drop_duplicates(subset=['Smiles'])
    return data_main
        

In [7]:
# store result of data processing in df_main
df_main = process_data(file_names)

In [8]:
df_main

,Molecule ChEMBL ID,Smiles,Standard Value
0,CHEMBL3647059,C[C@H]1CC[C@H](Cn2c(N3CCO[C@H]4CCC[C@@H]43)nc3...,7.229148
1,CHEMBL3649823,COC(C)C1CCCN1c1nc2cc(-c3noc(=O)[nH]3)nc(-c3cnc...,6.017277
2,CHEMBL3649822,C[C@H]1CC[C@H](Cn2c(N3CCOCC3c3ccccc3F)nc3cc(-c...,8.096910
3,CHEMBL3649980,CC(F)(c1ccccc1F)c1nc2cc(-c3noc(=O)[nH]3)nc(-c3...,8.698970
4,CHEMBL3649984,Cc1cccnc1C(C)(F)c1nc2cc(-c3noc(=O)[nH]3)nc(-c3...,9.000000
...,...,...,...
351,CHEMBL3649846,C[C@H]1CC[C@H](Cn2c(N3CCCC34CCOC4)nc3cc(-c4noc...,8.221849
352,CHEMBL3968776,COC(=O)N1C[C@H](C)N(c2nc3cc(-c4nc(=O)o[nH]4)nc...,8.698970
353,CHEMBL3966104,CCCNC(=O)N1C[C@H](C)N(c2nc3cc(-c4nc(=O)o[nH]4)...,8.698970
354,CHEMBL3649964,CCCOC(CC)c1nc2cc(-c3noc(=O)[nH]3)nc(-c3cncc(Cl...,8.522879


In [9]:
df_main.describe()

,Standard Value
count,270.000000
mean,8.223129
std,0.664805
min,6.017277
25%,7.795880
50%,8.397940
75%,8.698970
max,9.000000


In [10]:
df_main.isna().describe()

,Molecule ChEMBL ID,Smiles,Standard Value
count,270,270,270
unique,1,1,1
top,False,False,False
freq,270,270,270


In [11]:
# save df_main to a csv file
df_main.to_csv('data_train.csv')

In [12]:
data = pd.read_csv('data_train.csv', index_col=0)

In [13]:
data.head()

,Molecule ChEMBL ID,Smiles,Standard Value
0,CHEMBL3647059,C[C@H]1CC[C@H](Cn2c(N3CCO[C@H]4CCC[C@@H]43)nc3...,7.229148
1,CHEMBL3649823,COC(C)C1CCCN1c1nc2cc(-c3noc(=O)[nH]3)nc(-c3cnc...,6.017277
2,CHEMBL3649822,C[C@H]1CC[C@H](Cn2c(N3CCOCC3c3ccccc3F)nc3cc(-c...,8.096910
3,CHEMBL3649980,CC(F)(c1ccccc1F)c1nc2cc(-c3noc(=O)[nH]3)nc(-c3...,8.698970
4,CHEMBL3649984,Cc1cccnc1C(C)(F)c1nc2cc(-c3noc(=O)[nH]3)nc(-c3...,9.000000
